# Практическая работа №3 «Электронная подпись (RSA, ГОСТы 34.10-94 и 34.10-2001)»

В практической работе необходимо привести последовательность выполнения процедур генерации и проверки ЭП с использованием следующих способов:
- на базе алгоритма RSA;
- по ГОСТ 34.10-94;
- по ГОСТ 34.10-2001.

При оформлении отчета необходимо привести таблицы генерации ключей, отправки сообщения с ЭП и получения сообщения с ЭП. В качестве хеш-образа исходного сообщения h(T) принять коды, соответственно, 1-ой, 2-ой и 3-ей буквы своей фамилии согласно их положению в алфавите. Отчет прикрепляется в edu.susu.ru

In [6]:
import json

import numpy as np

In [15]:
def power_with_modulo(a, n, m):
    res = 1
    while n > 0:
        if (n & 1) == 1:
            res = (res * a) % m
        a = (a * a) % m
        n >>= 1
    return res

In [7]:
with open('instance/settings.json') as f:
    settings = json.load(f)
    LAST_NAME = settings['last_name']

## Протокол на базе алгоритма RSA

### Этап 1. Выработка ключей 
Выполняет отправитель __А__

In [4]:
e, n = 139, 209
d = 79 

### Этап 2. Отправка сообщения и электронной подписи
Выполняет отправитель __А__

Вычисление хеш-образа $h = h(T)$, где $T$ – исходное сообщение, $h(T)$ – хеш-функция (для MD5 длина хеш-образа 128 бит).

In [20]:
h = int(''.join(map(lambda x: str(ord(x) - ord('a') + 1), LAST_NAME[:3])))
md = f'$h = {h}$'

{{md}}

Выработка цифровой подписи $s = h^d$ mod $n$, где $d$ – закрытый ключ отправителя **A**, $n$ – часть открытого ключа отправителя __A__

In [21]:
s = power_with_modulo(h, d, n)
md = f'$s = {s}$'

{{md}}

Отправка получателю __B__ исходного сообщения $Т$ и цифровой подписи $s$.

### Этап 3. Получение сообщения и проверка электронной подписи 
Выполняет получатель __B__

Вычисление хэш-образа по полученному сообщению $h^\prime = h(T^\prime)$, где $T^\prime$ – полученное сообщение.

In [25]:
h_prime = h
md = f'$h^\prime = {h_prime}$'

{{md}}

Вычисление хэш-образа из цифровой подписи  $h = s^e$ mod $n$, где $e$ и $n$ – открытый ключ отправителя __A__.

In [29]:
h = power_with_modulo(s, e, n)
md = f'$h = {h}$'

{{md}}

In [32]:
verified = h == (h_prime % n)

`{{'Verified' if verified else 'Not verified'}}`

Т.к. $h^\prime = h$, то получатель B делает вывод, что полученное сообщение $T^\prime = T$ и оно действительно отправлено __А__.

## Алгоритм цифровой подписи ГОСТ 34.10-94

### Этап 1. Выработка ключей
Выполняет отправитель __А__

Выбор $р$ - простого числа ($509 < p < 512$ битов, либо $1020 < p < 1024$ битов).

In [38]:
p = 79

Выбор $q$ - простого числа - множителя $(p - 1)$ ($254 < q < 256$ битов).

In [39]:
q = 13

Выбор $а$ - любого числа, меньшего $(p - 1)$, для которого $a^q$ mod $p = 1$.

In [40]:
a = 8

Выбор закрытого ключа $x$ - числа, меньшего $q$.

In [41]:
x = 4

Вычисление открытого ключа $y = a^x$ mod $p$.

In [48]:
y = power_with_modulo(a, x, p)
md = f'$y = {y}$'

{{md}}

Публикация ключей.  
Первые три параметра $р$, $q$ и $a$ - открыты и могут совместно использоваться пользователями сети,
$y$ – персональный открытый ключ для одного пользователя, $x$ – персональный закрытый ключ отправителя __А__.

### Этап 2. Отправка сообщения и электронной подписи
Выполняет отправитель __А__

Вычисление хеш-образа $h = h(T)$ (длина хеш-образа 256 бит).

In [50]:
h = int(''.join(map(lambda x: str(ord(x) - ord('a') + 1), LAST_NAME[:3])))
md = f'$h = {h}$'

{{md}}

Выбор $k$ - любого числа, меньшего $q$.

In [51]:
k = 11

Вычисление двух значений:  
$w = a^k$ mod $p$ и $w^\prime = w$ mod $q$ (длина $w^\prime$ 256 бит).  
Если $w^\prime = 0$, перейти к этапу 2 и выбрать другое значение числа k.  

In [53]:
w = power_with_modulo(a, k, p)
w_prime = w % q
md = f'$w = {w}, w^\prime = {w_prime}$'

{{md}}

Вычисление $s = (x*w^\prime + k*h)$ mod $q$ (длина $s$ 256 бит).  
Если $s = 0$, перейти к этапу 2 и выбрать другое значение числа $k$.

In [55]:
s = (x * w_prime + k * h) % q
md = f'$s = {s}$'

{{md}}

Отправка получателю __B__ исходного сообщения $Т$ и цифровой подписи $(w^\prime, s)$.

### Этап 3. Получение сообщения и проверка электронной подписи
Выполняет получатель __B__

Вычисление хеш-образа по полученному сообщению $h^\prime = h(T^\prime)$.  
Если $T = T^\prime$, то должно быть $h = h^\prime$.

In [59]:
h_prime = h
md = f'$h^\prime = {h_prime}$'

{{md}}

Вычисление $v = h^{\prime q-2}$ mod $q$.

In [61]:
v = power_with_modulo(h_prime, q - 2, q)
md = f'$v = {v}$'

{{md}}

Вычисление двух значений:   
$z_1 = (s*v)$ mod $q$ и $z_2 = ((q – w^\prime)* v)$ mod $q$.

In [63]:
z_1 = (s * v) % q
z_2 = ((q - w_prime) * v) % q
md = f'$z_1 = {z_1}, z_2 = {z_2}$'

{{md}}

Вычисление $u = ((a^{z_1} * y^{z_2})$ mod $p)$ mod $q$.

In [66]:
u = ((power_with_modulo(a, z_1, p) * power_with_modulo(y, z_2, p)) % p) % q
md = f'$u = {u}$'

{{md}}

In [67]:
verified = u == (w_prime % n)

`{{'Verified' if verified else 'Not verified'}}`

Т.к. $w^\prime = u$, то получатель **B** делает вывод, что полученное сообщение $T^\prime = T$ и оно действительно отправлено __А__.

## Алгоритм цифровой подписи ГОСТ Р 34.10-2001

In [187]:
def inverse_mod(k, p):
    """Returns the inverse of k modulo p.
    This function returns the only integer x such that (x * k) % p == 1.
    k must be non-zero and p must be a prime.
    """
    if k == 0:
        raise ZeroDivisionError('division by zero')

    if k < 0:
        # k ** -1 = p - (-k) ** -1  (mod p)
        return p - inverse_mod(-k, p)

    # Extended Euclidean algorithm.
    s, old_s = 0, 1
    t, old_t = 1, 0
    r, old_r = p, k

    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    gcd, x, y = old_r, old_s, old_t

    assert gcd == 1
    assert (k * x) % p == 1

    return x % p


def point_add(x1, y1, x2, y2, A, n):
    if x1 == x2 and y1 != y2:
        return None

    if x1 == x2:
        m = (3 * x1 * x1 + A) * inverse_mod(2 * y1, n)
    else:
        m = (y1 - y2) * inverse_mod(x1 - x2, n)

    x3 = m * m - x1 - x2
    y3 = y1 + m * (x3 - x1)
    result = (x3 % n, -y3 % n)
    return result

def point_mult(x1, y1, times, A, n):
    rx, ry = x1, y1
    for i in range(0, times - 1):
        rx, ry = point_add(rx, ry, x1, y1, A, n)
    return rx, ry

### Этап 1. Выработка ключей
Выполняет отправитель __А__

Выбирается модуль эллиптической кривой - простое число $n$ $(n > 2^{255})$.

In [176]:
n = 41

Выбираются коэффициенты эллиптической кривой $A$ и $B$.  
Должно соблюдаться условие $(4*A^3 + 27*B^2) \bmod n \ne 0$, в противном случае меняются параметры эллиптической кривой $n$, $A$ или $B$.

In [177]:
A = 3
B = 7

Определяется точка эллиптической кривой $P(x_p, y_p)$ и порядок циклической подгруппы группы точек эллиптической кривой $q$).

In [178]:
x_p = 7
y_p = 17
q = 47

Выбирается закрытый ключ $d$ $(0 < d < q)$.

In [179]:
d = 10

Определяется точка эллиптической кривой $Q(x_q, y_q)$: $Q(x_q, y_q) = d * P(x_p, y_p)$.

In [189]:
x_q, y_q = point_mult(x_p, y_p, d, A, n)
md = f'$Q(x_q, y_q) = Q({x_q}, {y_q})$'

{{md}}

Публикуется открытый ключ $[(A, B), P(x_p, y_p), n, Q(x_q, y_q)]$ в специальном хранилище, где исключается возможность его подмены (общедоступном сертифицированном справочнике).  
Для выработки и проверки электронной цифровой подписи $q$ является частью открытого ключа вместо $n$.

### Этап 2. Отправка сообщения и электронной подписи
Выполняет отправитель __А__

Вычисление хеш-образа $h = h(T)$ (длина хеш-образа 256 бит).

In [96]:
h = int(''.join(map(lambda x: str(ord(x) - ord('a') + 1), LAST_NAME[:3])))
md = f'$h = {h}$'

{{md}}

Вычисление $e = h \bmod q$, где $q$ – часть открытого ключа отправителя __A__.

In [101]:
e = h % q
md = f'$e = {e}$'

{{md}}

Выбор $k$ - любого числа, меньшего $q$.

In [110]:
k = 11

Определение точки эллиптической кривой $C(x_c, y_c) = k*P(x_p, y_p)$, где $P(x_p, y_p)$ – часть открытого ключа отправителя __A__.

In [192]:
x_c, y_c = point_mult(x_p, y_p, k, A, n)
md = f'$C(x_c, y_c) = C({x_c}, {y_c})$'

{{md}}

Вычисление $r = x_c \bmod q$.  
Если $r = 0$, перейти к этапу 2 и выбрать другое значение числа $k$.

In [197]:
r = x_c % q
md = f'$r = {r}$'

{{md}}

Вычисление $s = (r*d + k*e) \bmod q$, где $d$ - закрытый ключ отправителя **A**.  
Если $s = 0$, перейти к этапу 2 и выработать другое значение числа $k$.

In [201]:
s = (r * d + k * e) % q
md = f'$s = {s}$'

{{md}}

Отправка получателю **B** исходного сообщения $T$ и цифровой подписи $(r, s)$.

### Этап 3. Получение сообщения и проверка электронной подписи
Выполняет получатель **B**

Вычисление хеш-образа по полученному сообщению $h^\prime = h(T^\prime)$.

In [202]:
h_prime = h
md = f'$h^\prime = {h_prime}$'

{{md}}

Вычисление $e^\prime = h^\prime \bmod q$.

In [204]:
e_prime = h_prime % q
md = f'$e^\prime = {e_prime}$'

{{md}}

Вычисление $v = e^{\prime - 1} \bmod q$.

In [208]:
v = inverse_mod(e, q)
md = f'$v = {v}$'

{{md}}

Вычисление двух значений:  
$z_1 = (s*v) \bmod q$ и $z_2 = ((q - r)*v) \bmod q$.

In [211]:
z_1 = (s * v) % q
z_2 = ((q - r) * v) % q
md = f'$z_1 = {z_1}, z_2 = {z_2}$'

{{md}}

Определение точки эллиптической кривой $C^\prime(x_{c^\prime}, y_{c^\prime}) = z_1*P(x_p, y_p) + z_2*Q(x_q, y_q)$, где $Q(x_q, y_q)$ – часть открытого ключа отправителя **A**.

In [216]:
m_1 = point_mult(x_p, y_p, z_1, A, n)
m_2 = point_mult(x_q, y_q, z_2, A, n)

x_c_prime, y_c_prime = point_add(*m_1, *m_2, A, n)
md = f'$C^\prime(x_{{c^\prime}}, y_{{c^\prime}}) = C^\prime({x_c_prime}, {y_c_prime})$'

{{md}}

Вычисление $r^\prime = x_{c^\prime} \bmod q$.

In [219]:
r_prime = x_c_prime % q
md = f'$r^\prime = {r_prime}$'

{{md}}

In [225]:
verified = r == r_prime

`{{'Verified' if verified else 'Not verified'}}`

Т.к. $r^\prime = r$, то получатель B делает вывод, что полученное сообщение $T^\prime = T$ и оно действительно отправлено **А**.